In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "../../data/Landslide2.csv"
device = "102"
save_path = f"../../data/dev{device}_resample1T.csv"
#save_path = f"data/devAll2_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo


df = df.drop(columns=['unixt', 'gateway time', 'old time', 'risk_level', 'confidence'], errors='ignore')
df = df[df['devID'] != 218499049.0]
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] != 0.0]
df = df[df['rain'] != -1.0]

df = df[df['devID'] == int(device)]
df

C:\Users\ahmad\AppData\Local\Temp\ipykernel_10032\3855835330.py:12: DtypeWarning: Columns (9,10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
1,2025-05-02 11:42:00.384119,102.0,22.200001,0.0,34.549099,61.175941,3.323395,8.637964,99.898933
3,2025-05-02 11:42:11.034615,102.0,22.200001,0.0,34.506374,61.542152,14.766250,8.637963,99.898933
5,2025-05-02 11:42:21.675224,102.0,22.200001,0.0,34.485012,61.767223,3.242806,8.637963,99.898933
7,2025-05-02 11:42:32.443581,102.0,22.200001,0.0,34.477001,62.366138,6.375742,8.637964,99.898933
9,2025-05-02 11:42:43.142656,102.0,22.200001,0.0,34.447620,60.361488,19.010500,8.637964,99.898933
...,...,...,...,...,...,...,...,...,...
308532,2025-06-29 20:24:40.220000,102.0,25.600000,0.0,29.045549,81.918671,15.166560,8.637934,99.898949
308533,2025-06-29 20:24:51.870000,102.0,25.600000,0.0,29.862669,77.020523,17.665491,8.637932,99.898941
308535,2025-06-29 20:25:14.920000,102.0,25.600000,0.0,29.993513,76.671471,2.676137,8.637932,99.898941
308537,2025-06-29 20:25:48.980000,102.0,25.600000,0.0,29.908064,77.426796,3.856516,8.637931,99.898941


In [9]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
count,151584,151584.0,151584.000000,151584.000000,151584.000000,151584.000000,151584.000000,151584.000000,151584.000000
mean,2025-06-02 20:08:53.237193472,102.0,49.656115,0.191383,29.985297,74.800888,4.983740,8.637946,99.898928
min,2025-05-02 11:42:00.384119,102.0,21.100000,0.000000,24.238958,43.418247,0.000000,8.637330,99.883331
25%,2025-05-08 06:00:44.477559552,102.0,43.400002,0.000000,27.331200,67.895622,2.681147,8.637937,99.898918
50%,2025-06-06 00:59:50.781526016,102.0,52.299999,0.000000,28.540855,79.240715,3.896273,8.637945,99.898933
75%,2025-06-17 06:26:38.380000256,102.0,59.000000,0.000000,33.208591,81.386513,3.961901,8.637955,99.898941
max,2025-06-29 20:26:01.100000,102.0,64.500000,34.366199,38.984512,88.510567,32.062031,8.638361,99.900078
std,NaN,0.0,11.465118,1.827632,3.316074,9.083173,4.683018,0.000021,0.000049


In [10]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
#df = df.sort_values(['devID', 'timestamp']).reset_index(drop=True)

# ตั้ง timestamp เป็น index
df = df.set_index('timestamp')
df

,devID,soil,rain,temp,humi,geo,lat,lng
timestamp,,,,,,,,
2025-05-02 11:42:00.384119,102.0,22.200001,0.0,34.549099,61.175941,3.323395,8.637964,99.898933
2025-05-02 11:42:11.034615,102.0,22.200001,0.0,34.506374,61.542152,14.766250,8.637963,99.898933
2025-05-02 11:42:21.675224,102.0,22.200001,0.0,34.485012,61.767223,3.242806,8.637963,99.898933
2025-05-02 11:42:32.443581,102.0,22.200001,0.0,34.477001,62.366138,6.375742,8.637964,99.898933
2025-05-02 11:42:43.142656,102.0,22.200001,0.0,34.447620,60.361488,19.010500,8.637964,99.898933
...,...,...,...,...,...,...,...,...
2025-06-29 20:24:40.220000,102.0,25.600000,0.0,29.045549,81.918671,15.166560,8.637934,99.898949
2025-06-29 20:24:51.870000,102.0,25.600000,0.0,29.862669,77.020523,17.665491,8.637932,99.898941
2025-06-29 20:25:14.920000,102.0,25.600000,0.0,29.993513,76.671471,2.676137,8.637932,99.898941


In [ ]:
window = '1T'  # 10 นาที
features = ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']


df_rolling = df[features].resample(window).agg(['mean', 'std', 'min', 'max'])
df_rolling.columns = ['_'.join(col) for col in df_rolling.columns]
df_rolling['devID'] = int(device)
df_rolling = df_rolling.dropna()

df_rolling = df_rolling.reset_index()
df_rolling = df_rolling[['timestamp', 'devID', 'soil_mean', 'rain_mean', 'temp_mean', 'humi_mean', 'geo_mean', 'geo_std']]
df_rolling

C:\Users\ahmad\AppData\Local\Temp\ipykernel_10032\1659244873.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_rolling = df[features].resample(window).agg(['mean', 'std', 'min', 'max'])


,timestamp,devID,soil_mean,rain_mean,temp_mean,humi_mean,geo_mean,geo_std
0,2025-05-02 11:40:00,102,22.200001,0.0,34.717920,62.513730,7.895913,5.625963
1,2025-05-02 11:50:00,102,22.203572,0.0,34.847077,61.514394,9.278016,7.178730
2,2025-05-02 12:00:00,102,22.200001,0.0,35.709997,61.343821,9.139770,6.965634
3,2025-05-02 12:10:00,102,22.203704,0.0,34.937530,62.299312,7.868829,6.421354
4,2025-05-02 12:20:00,102,22.125000,0.0,34.723956,63.180795,8.939695,5.708089
...,...,...,...,...,...,...,...,...
3331,2025-06-29 19:40:00,102,25.600000,0.0,29.510186,80.122087,5.407425,4.375503
3332,2025-06-29 19:50:00,102,25.600000,0.0,29.486474,80.045778,5.613575,3.926898
3333,2025-06-29 20:00:00,102,25.600000,0.0,29.724941,78.549214,8.002298,5.757017
3334,2025-06-29 20:10:00,102,25.600000,0.0,29.742026,78.720270,8.012238,5.920143


In [12]:
df_rolling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3336 entries, 0 to 3335
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  3336 non-null   datetime64[ns]
 1   devID      3336 non-null   int64         
 2   soil_mean  3336 non-null   float64       
 3   rain_mean  3336 non-null   float64       
 4   temp_mean  3336 non-null   float64       
 5   humi_mean  3336 non-null   float64       
 6   geo_mean   3336 non-null   float64       
 7   geo_std    3336 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 208.6 KB


In [13]:
nan_counts = df_rolling.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [14]:

df_rolling.to_csv(f"{save_path}", index=False)

print("✅ บันทึก เรียบร้อยแล้ว", save_path)

✅ บันทึก เรียบร้อยแล้ว ../../data/dev102_resample10T.csv
